# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [233]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Imports here
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict

from datetime import datetime

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [53]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [54]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = {"train": transforms.Compose([transforms.Resize(255), transforms.RandomVerticalFlip(),
                                                transforms.RandomHorizontalFlip(),  
                                                transforms.CenterCrop(224), transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                   "test": transforms.Compose([transforms.Resize(255), transforms.CenterCrop(224), transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                   "valid": transforms.Compose([transforms.Resize(255),transforms.CenterCrop(224),transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

# TODO: Load the datasets with ImageFolder
image_datasets = {"train": datasets.ImageFolder(train_dir, transform=data_transforms["train"]),
                  "test": datasets.ImageFolder(test_dir, transform=data_transforms["test"]),
                  "valid": datasets.ImageFolder(valid_dir, transform=data_transforms["valid"])}
                  
# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {"train": torch.utils.data.DataLoader(image_datasets["train"], batch_size=4, shuffle=True),
               "test": torch.utils.data.DataLoader(image_datasets["test"], batch_size=4, shuffle=True),
               "valid": torch.utils.data.DataLoader(image_datasets["valid"], batch_size=4, shuffle=True)}


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [56]:
import json

# read json-data and convert it into a dict 

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    cat_to_name_dict = dict()
    for key in cat_to_name: #
        cat_to_name_dict[int(key)] = cat_to_name[key]
    print(cat_to_name_dict)
    

{21: 'fire lily', 3: 'canterbury bells', 45: 'bolero deep blue', 1: 'pink primrose', 34: 'mexican aster', 27: 'prince of wales feathers', 7: 'moon orchid', 16: 'globe-flower', 25: 'grape hyacinth', 26: 'corn poppy', 79: 'toad lily', 39: 'siam tulip', 24: 'red ginger', 67: 'spring crocus', 35: 'alpine sea holly', 32: 'garden phlox', 10: 'globe thistle', 6: 'tiger lily', 93: 'ball moss', 33: 'love in the mist', 9: 'monkshood', 102: 'blackberry lily', 14: 'spear thistle', 19: 'balloon flower', 100: 'blanket flower', 13: 'king protea', 49: 'oxeye daisy', 15: 'yellow iris', 61: 'cautleya spicata', 31: 'carnation', 64: 'silverbush', 68: 'bearded iris', 63: 'black-eyed susan', 69: 'windflower', 62: 'japanese anemone', 20: 'giant white arum lily', 38: 'great masterwort', 4: 'sweet pea', 86: 'tree mallow', 101: 'trumpet creeper', 42: 'daffodil', 22: 'pincushion flower', 2: 'hard-leaved pocket orchid', 54: 'sunflower', 66: 'osteospermum', 70: 'tree poppy', 85: 'desert-rose', 99: 'bromelia', 87: 

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [62]:
def create_model(input_size, hidden_size, output_size, dropout = 0.5, state_dict = None):
    model = models.vgg16(pretrained=True)
    # build classifier at the end of the pretrained
    for param in model.parameters():
        param.requieres_grad = False # Disable optimizer on pretrained network
    
    model.classifier = create_classifier(input_size, hidden_size, output_size)  # replace old classifier sequence with our new one
    
    if not state_dict == None:                               #load state_dict, if available
         model.load_state_dict(state_dict)
    
    return model



def create_classifier(input_size, hidden_size, output_size, dropout = 0.5):
    """Creates classifer for the model."""
    
    
    layer_size = zip(hidden_size[:-1], hidden_size[1:])
    
    sequence = []

    sequence.append(["fc_in",nn.Linear(input_size,hidden_size[0])])
    sequence.append(["relu_in",nn.ReLU(True)])
    sequence.append(["drop_in",nn.Dropout(dropout)])

    for i, (l1, l2) in enumerate(layer_size):
        sequence.append(["fc"+str(i+1),nn.Linear(l1,l2)])
        sequence.append(["relu"+str(i+1),nn.ReLU(True)])
        sequence.append(["drop"+str(i+1),nn.Dropout(dropout)])

    sequence.append(["fc_out",nn.Linear(hidden_size[-1],output_size)])
    sequence.append(["soft_out",nn.LogSigmoid()])

    return nn.Sequential(OrderedDict(sequence))

In [63]:
# TODO: Build and train your network
input_size = 25088
hidden_size = [1024, 512]
output_size = max(cat_to_name_dict)
dropout = 0.5

model = create_model(input_size, hidden_size, output_size, dropout = 0.5)

criterion = nn.CrossEntropyLoss() # 
optimizer = optim.SGD(model.classifier.parameters(), lr=0.005) # just optimize OUR classifier with learnrate = 0.001

model #print model (debug)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [70]:
def train_model(model, train_dataloader, test_dataloader, optimizer, criterion, epochs = 3, print_sequence = 50):
    """TODO DocStr: Train model"""
    steps = 0

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # look if cuda is available
    model.to(device)

    time_start = datetime.now() # set start-timestamp
    
    for e in range(epochs):
        model.train() # set model in training-mode
        running_loss = 0
        for images, labels in iter(train_dataloader): # loop through training-images
            steps += 1
            
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad() # reset gradient

            # Forward and backward passes
            with torch.set_grad_enabled(True): # make sure gradient decent is enabled
                outputs = model.forward(images) # feed forward batch of images
                loss = criterion(outputs, labels) # get loss
                loss.backward() # calculate gradient decent
                optimizer.step() # optimize weights

            running_loss += loss.item() 

            if steps % print_sequence == 0:
                validation = validate(model, test_dataloader, criterion, device)
                duration = datetime.now() - time_start # calculate time since start of training 
                print("{} - Iterations: {} - Epoch: {} - Loss: {} - Accuracy: {}".format(duration, steps, e+1, running_loss / print_sequence, 
                                                                               validation))
                running_loss = 0
    print("Done.")

In [69]:
# TODO: Do validation on the valid set
def validate(model, val_dataloader, criterion, device):
    """Validates trained network"""
    
    model.to(device) # calculate with CUDA, if available
    model.eval() # set model to evaluation mode -> no gradient decent
    
    test_loss = 0
    accuracy = 0
    steps = 0
    with torch.no_grad(): # make sure that gradient decent is deactivated
        for images, labels in iter(val_dataloader): # loop through validation-pictures
            steps += 1
            
            images, labels = images.to(device), labels.to(device)

            output = model.forward(images) # feed batch of images through network
            test_loss += criterion(output, labels).item() # calculate total loss of test-routine
            
            ps = torch.exp(output) #propabilities 
            equal = (ps.max(dim=1)[1] == labels.data) # check if pictures is classified correctly
            accuracy += equal.type(torch.FloatTensor).mean() # calculate total accuracy
        #test_loss = test_loss / steps
        accuracy = accuracy / steps # calculate mean accuracy
    return accuracy.item()

In [71]:
train_model(model, dataloaders["train"], dataloaders["valid"], optimizer, criterion, print_sequence = 100)

0:00:55.151527 - Iterations: 100 - Epoch: 1 - Loss: 4.517947092056274 - Accuracy: 0.07439024746417999
0:01:50.089794 - Iterations: 200 - Epoch: 1 - Loss: 4.38914541721344 - Accuracy: 0.09024390578269958
0:02:45.242994 - Iterations: 300 - Epoch: 1 - Loss: 4.275572996139527 - Accuracy: 0.04878048598766327
0:03:40.347976 - Iterations: 400 - Epoch: 1 - Loss: 4.155853281021118 - Accuracy: 0.03292682766914368
0:04:35.365977 - Iterations: 500 - Epoch: 1 - Loss: 3.934931044578552 - Accuracy: 0.03292682766914368
0:05:30.402933 - Iterations: 600 - Epoch: 1 - Loss: 3.651284136772156 - Accuracy: 0.25731706619262695
0:06:25.477803 - Iterations: 700 - Epoch: 1 - Loss: 3.2608378744125366 - Accuracy: 0.22439023852348328
0:07:20.499310 - Iterations: 800 - Epoch: 1 - Loss: 2.9775637459754942 - Accuracy: 0.30040648579597473
0:08:15.462478 - Iterations: 900 - Epoch: 1 - Loss: 2.6013788419961927 - Accuracy: 0.32032519578933716
0:09:10.597881 - Iterations: 1000 - Epoch: 1 - Loss: 2.3510422670841216 - Accura

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [92]:
def test_model(model, test_dataloader, criterion, device):

    model.to(device)
    model.eval()
    
    test_loss = 0
    accuracy = 0
    steps = 0
    with torch.no_grad():
        for images, labels in iter(test_dataloader):
            steps += 1
            
            images, labels = images.to(device), labels.to(device)

            output = model.forward(images)
            test_loss += criterion(output, labels).item()
            
            ps = torch.exp(output) #propabilities
            equal = (ps.max(dim=1)[1] == labels.data) # check if pictures is classified correctly
            accuracy += equal.type(torch.FloatTensor).mean()
        #test_loss = test_loss / steps
        accuracy = accuracy / steps
    return accuracy.item()

In [93]:
#test
test_model(loading_model, dataloaders["test"], criterion, "cuda:0" )

0.7617886662483215

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [74]:
# TODO: Save the checkpoint 
def save_model(model, input_size, hidden_size, output_size):
    checkpoint = {"input_size": input_size,
                  "output_size": output_size,
                  "hidden_layers": hidden_size,
                  "state_dict": model.state_dict()}
    torch.save(checkpoint, 'checkpoint_1024_512_logsigmoid.pth')
    #print(checkpoint["state_dict"].keys())
    print("Saved.")

In [82]:
save_model(model, input_size, hidden_size, output_size)

Saved.


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [83]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_model(filepath):
    checkpoint = torch.load(filepath)
    return create_model(checkpoint["input_size"], checkpoint["hidden_layers"], 
                 checkpoint["output_size"], state_dict = checkpoint["state_dict"] )

In [94]:
#test load model
model = load_model('checkpoint_1024_512_logsigmoid.pth')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [130]:
from PIL import Image
import numpy as np

In [270]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    x,y = image.size
    
    if x < y:    #get x,y for thumbnail
        x_thumb = 256
        y_thumb = int(x_thumb * y/x)
    elif x > y:
        y_thumb = 256
        x_thumb = int(y_thumb * x/y)
    else:
        x_thumb = 256
        y_thumb = 256
    
    image.thumbnail((x_thumb,y_thumb), Image.ANTIALIAS)
    
    x1 = x_thumb / 2 - 224 / 2 # center crop image 224x224
    x2 = x1 + 224
    y1 = y_thumb / 2 - 224 / 2
    y2 = y1 + 224

    
    image = image.crop((x1, y1, x2, y2))
    
    np_image = np.array(image,dtype=np.float32) # convert to np array
    
    means = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
   
    np_image = ((np_image / 255) - means) / std # normalize image
    #print(np_image.dtype)
    
    return np_image.transpose(2,0,1).astype(np.float32) # transpose ndarray
    
    

In [271]:
#debug test

image = Image.open("buttercup_flower_test.jpg")
im = process_image(image)
#print(im.dtype)

imshow(im)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [187]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [278]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    image = process_image(Image.open(image_path))
    image_tensor = torch.from_numpy(image)
    image_tensor.type(torch.DoubleTensor)
    
    model.eval()
    with torch.no_grad():
        output = model.forward(image_tensor.unsqueeze(0))
    top_k = torch.topk(output, topk)
    print(top_k)
    
    #return probs , classes
    # TODO: Implement the code to predict the class from an image file

In [281]:
#test:
predict("buttercup_flower_test.jpg", model, topk=5)

(tensor([[ -0.1489, -11.9220, -12.3447, -12.7597, -12.8633]]), tensor([[ 45,   9,  51,  57,  10]]))


## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes